## 資料處理

In [1]:
import numpy as np
import json
import os

In [2]:
root_path="D:/DeepLearning-workplace/taiko_generate/preprocessing/audio_np"
audio_path = os.listdir(root_path)
audio_path = [ root_path+"/"+ad for ad in audio_path]

root_path="D:/DeepLearning-workplace/taiko_generate/preprocessing/map_np"
map_path = os.listdir(root_path)
map_path = [ root_path+"/"+mp for mp in map_path]

In [3]:
class data_generator:
    def __init__(self, audio_path, map_path, batch_size=64):
        self.audio_path = audio_path
        self.map_path = map_path
        self.batch_size = batch_size
        data_len=0
        self.steps=9508
#         for audio in self.audio_path:
#             audio = np.load(audio)
#             data_len+=len(audio)
#         self.steps =data_len // self.batch_size
#         if data_len % self.batch_size != 0:
#             self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            for i, audio in enumerate(self.audio_path): #所有的音檔path
                audio = np.load(audio) #讀音檔
                f = open(self.map_path[i], "r") #讀所有的map
                maps = json.load(f)
                f.close()

                for key in maps.keys(): #讀所有難度的map
                    step=1 #紀錄batch step
                    for idx in range(0,len(audio),self.batch_size): #每次取batch_size大小的資料出來
                        if idx+self.batch_size > len(audio):
                            X = np.array(audio[idx:len(audio)]) #cnn input
                        else:
                             X = np.array(audio[idx:idx+self.batch_size])
                        C = np.zeros((len(X),5)) #condition
                        Y = np.zeros(len(X)) #output
                        C[:,int(key)]=1

                        n=0 #已被記錄過的數量
                        for t,_ in maps[key]:
                            if (t-7)>step*self.batch_size: #音檔前70ms不取
                                break
                            else:
                                Y[t-(step-1)*self.batch_size-8]=1
                                n+=1
                        for i in range(n): #把紀錄過的資料pop掉
                            maps[key].pop(0)
                        step+=1
                        yield [X, C], Y

In [4]:
# class data_generator:
#     def __init__(self, audio_path, map_path, batch_size=64):
#         self.audio_path = audio_path
#         self.map_path = map_path
#         self.batch_size = batch_size
#         data_len=0
#         #self.steps=7782
#         for audio in self.audio_path:
#             audio = np.load(audio)
#             data_len+=len(audio)
#         self.steps =data_len // self.batch_size
#         if data_len % self.batch_size != 0:
#             self.steps += 1
#     def __len__(self):
#         return self.steps
#     def __iter__(self):
#         while True:
#             for i, audio in enumerate(audio_path): #所有的音檔path
#                 audio = np.load(audio) #讀音檔
#                 f = open(map_path[i], "r") #讀所有的map
#                 maps = json.load(f)
#                 f.close()

#                 steps=len(audio)//self.batch_size
#                 for key in maps.keys(): #讀所有難度的map
#                     for _ in range(steps):
#                         t_idxs = list(range(len(maps[key])))#產生打點亂數idx
#                         np.random.shuffle(t_idxs)
#                         t_idxs=[maps[key][i][0]-8 for i in t_idxs]

#                         idxs = list(range(len(audio))) #產生亂數idx
#                         np.random.shuffle(idxs)
#                         idxs = idxs[:self.batch_size//2]

#                         Y = np.zeros(self.batch_size) #產生Y
#                         for i, idx in enumerate(idxs):
#                             if idx in t_idxs: #檢查亂數idx是否有打點
#                                 Y[i]=1
#                         Y[self.batch_size//2:]=1

#                         t_idxs=t_idxs[:self.batch_size//2]
#                         X = [audio[i] for i in idxs] #產生X
#                         for t_idx in t_idxs:
#                             X.append(audio[t_idx])
#                         X=np.array(X)

#                         C = np.zeros((self.batch_size,5)) #condition
#                         C[:,int(key)]=1

#                         yield [X, C], Y

In [3]:
t_audio_path = audio_path[:int(len(audio_path)*0.9)]
t_map_path = map_path[:int(len(map_path)*0.9)]

v_audio_path = audio_path[int(len(audio_path)*0.9):]
v_map_path = map_path[int(len(map_path)*0.9):]

print(len(t_audio_path),len(t_map_path),len(v_audio_path),len(v_map_path))

151 151 17 17


In [6]:
train_D = data_generator(t_audio_path,t_map_path,256)
print(train_D.__len__())
#valid_D =  data_generator(v_audio_path,v_map_path,256)
#print(valid_D.__len__())
# train_D = data_generator(audio_path,map_path,128)
# print(train_D.__len__())

9508


## 模型建立

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MultiHeadAttention, LayerNormalization, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

In [8]:
inputs=Input(shape=(15,80,3))
x = MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
x = LayerNormalization(epsilon=1e-6)(x)
x = MultiHeadAttention(num_heads=2, key_dim=2)(x, x)
x = LayerNormalization(epsilon=1e-6)(x)
flatten_layer=Flatten()
x = flatten_layer(inputs)

diffs = Input(shape=(5))
x = Concatenate(axis=1)([x,diffs])
x = Dense(256,activation='sigmoid')(x)
x = Dense(128,activation='sigmoid')(x)
outputs = Dense(1,activation='sigmoid')(x)

model=Model([inputs,diffs],outputs)

In [9]:
def loss_fun(y_true, y_pred):
    return K.binary_crossentropy(y_pred,y_true)#+y_true*K.binary_crossentropy(y_pred,y_true)

mc = ModelCheckpoint(
    'D:/DeepLearning-workplace/taiko_generate/StepPlacement/model_checkpoints/attention_model{epoch:03d}-loss{loss:.3f}.h5', 
    monitor='loss', 
    save_best_only=True, 
)

In [10]:
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 15, 80, 3)]  0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 3600)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 3605)         0           flatten[0][0]                    
                                                                 input_2[0][0]                

In [11]:
history = model.fit(
    train_D.__iter__(),
    steps_per_epoch=train_D.__len__(),
    epochs=100,
    callbacks=[mc]
)

Epoch 1/100
9508/9508 [==============================] - 159s 16ms/step - loss: 0.1530 - accuracy: 0.9615
Epoch 2/100
9508/9508 [==============================] - 157s 17ms/step - loss: 0.1308 - accuracy: 0.9623
Epoch 3/100
9508/9508 [==============================] - 165s 17ms/step - loss: 0.1307 - accuracy: 0.9618
Epoch 4/100
9508/9508 [==============================] - 166s 17ms/step - loss: 0.1268 - accuracy: 0.9611
Epoch 5/100
9508/9508 [==============================] - 149s 16ms/step - loss: 0.1274 - accuracy: 0.9619
Epoch 6/100
9508/9508 [==============================] - 154s 16ms/step - loss: 0.1311 - accuracy: 0.9606
Epoch 7/100
9508/9508 [==============================] - 162s 17ms/step - loss: 0.1113 - accuracy: 0.9658
Epoch 8/100
9508/9508 [==============================] - 161s 17ms/step - loss: 0.1230 - accuracy: 0.9609
Epoch 9/100
9508/9508 [==============================] - 150s 16ms/step - loss: 0.1295 - accuracy: 0.9597
Epoch 10/100
9508/9508 [======================

In [12]:
model.save("D:/DeepLearning-workplace/taiko_generate/model/StepPlacementAttention100.h5")

In [13]:
import pickle
def save_history(history, fn):
    with open(fn, 'wb') as fw:
        pickle.dump(history.history, fw, protocol=2)
save_history(history,"D:/DeepLearning-workplace/taiko_generate/StepPlacement/attention_history.bin")

## 測試

In [4]:
from tensorflow import keras
import librosa
from librosa.util import peak_pick
import matplotlib.pyplot as plt
from scipy import signal

In [5]:
model = keras.models.load_model("D:/DeepLearning-workplace/taiko_generate/model/StepPlacementAttention100.h5")

In [ ]:
F_measure_list=[]
for i,ap in enumerate(v_audio_path):
    audio=np.load(ap)
    f = open(v_map_path[i], "r") #讀所有的map
    maps = json.load(f)
    f.close()
    
    for key in maps.keys():
        
        dif = np.zeros(shape=(audio.shape[0],5))
        dif[:,int(key)]=1
        result = model.predict([audio, dif])

        data = []
        for r in result:
            data.append(r[0])
        data=np.array(data)

        data = data*1000
        data = data.astype(int)

        win = signal.windows.hamming(50)
        x = signal.convolve(data,win,mode='same')/sum(win)

        peaks, _ = signal.find_peaks(x, prominence=6-int(key))
        
        
        
        # 取出實際map之節拍毫秒值
        realMiliSecs = list()
        for ele in maps[key]:
            realMiliSecs.append(ele[0])

        # 誤差區間值
        DET_RANGE = 10
        # 計算TP, FP, FN score以得出精準度
        tpScore, fpScore, fnScore, previousRealMiliSecs, hitFlag = 0, 0, 0, 0, False
        for guessMiliSecs in peaks:
            # 取得與猜測時間點最近的實際時間點
            closestRealMiliSecs = min(realMiliSecs, key = lambda x:abs(x-guessMiliSecs))
            if previousRealMiliSecs != closestRealMiliSecs:
                if not hitFlag:
                    fnScore += 1
                hitFlag = False
            #print([closestRealMiliSecs, guessMiliSecs])
            if (closestRealMiliSecs - 8) + DET_RANGE > guessMiliSecs > (closestRealMiliSecs - 8) - DET_RANGE:
                #print('hit')
                tpScore += 1
                hitFlag = True
            else:
                fpScore += 1
            previousRealMiliSecs = closestRealMiliSecs

        F_measure = 2 * tpScore / (2 * tpScore + fpScore + fnScore)
        F_measure_list.append(F_measure)
        print(F_measure)

In [ ]:
print(sum(F_measure_list)/len(F_measure_list))

In [ ]:
#epoch:97 loss:0.091 F-measure:0.6403496725566742